In [1]:
%load_ext autoreload
%autoreload 2

In [81]:
from gpt3forchem.data import get_bandgap_data
from gpt3forchem.api_wrappers import fine_tune, query_gpt3, extract_inverse_prediction
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import time

from fastcore.helpers import save_pickle
import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

In [3]:
data = get_bandgap_data()

In [4]:
train_data_small, test_data_small = train_test_split(
    data, train_size=500, test_size=100, random_state=42
)

In [5]:
smiles_selfies = dict(zip(data["smiles"], data["selfies"]))

In [6]:
data = data.groupby("smiles").agg("mean").reset_index()
data["selfies"] = data["smiles"].map(smiles_selfies)

In [8]:
PROMPT_TEMPLATE_bandgap_inverse = "What is a molecule with a bandgap of {} eV###"
COMPLETION_TEMPLATE_bandgap_inverse = "{}@@@"


def generate_inverse_photoswitch_prompts(
    data: pd.DataFrame, representation: str = "smiles"
) -> pd.DataFrame:
    prompts = []
    completions = []
    smiles = []
    for i, row in data.iterrows():

        prompt = PROMPT_TEMPLATE_bandgap_inverse.format(
            np.round(row["GFN2_HOMO_LUMO_GAP"] * 27.2114, 1)
        )

        completion = COMPLETION_TEMPLATE_bandgap_inverse.format(row[representation])
        prompts.append(prompt)
        completions.append(completion)
        smiles.append(row["smiles"])

    prompts = pd.DataFrame(
        {"prompt": prompts, "completion": completions, "SMILES": smiles}
    )

    return prompts

In [11]:
(
    train_prompts_small_smiles,
    test_prompts_small_smiles,
) = generate_inverse_photoswitch_prompts(
    train_data_small
), generate_inverse_photoswitch_prompts(
    test_data_small
)

In [13]:
def train_inverse_model(train_prompts, representation):

    train_size = len(train_prompts)

    filename_base = time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())
    train_filename = f"run_files/{filename_base}_train_prompts_bandgap_inverse_{representation}_{train_size}.jsonl"

    train_prompts.to_json(train_filename, orient="records", lines=True)

    modelname = fine_tune(train_filename, train_filename, "ada")
    return modelname, train_filename

In [14]:
modelname_small_smiles, train_filename_small_smiles = train_inverse_model(
    train_prompts_small_smiles, "smiles"
)

Traceback (most recent call last):
  File "/Users/kevinmaikjablonka/miniconda3/envs/gpt3/bin/openai", line 8, in <module>
    sys.exit(main())
  File "/Users/kevinmaikjablonka/miniconda3/envs/gpt3/lib/python3.9/site-packages/openai/_openai_scripts.py", line 63, in main
    args.func(args)
  File "/Users/kevinmaikjablonka/miniconda3/envs/gpt3/lib/python3.9/site-packages/openai/cli.py", line 545, in sync
    resp = openai.wandb_logger.WandbLogger.sync(
  File "/Users/kevinmaikjablonka/miniconda3/envs/gpt3/lib/python3.9/site-packages/openai/wandb_logger.py", line 74, in sync
    fine_tune_logged = [
  File "/Users/kevinmaikjablonka/miniconda3/envs/gpt3/lib/python3.9/site-packages/openai/wandb_logger.py", line 75, in <listcomp>
    cls._log_fine_tune(
  File "/Users/kevinmaikjablonka/miniconda3/envs/gpt3/lib/python3.9/site-packages/openai/wandb_logger.py", line 125, in _log_fine_tune
    wandb_run = cls._get_wandb_run(run_path)
  File "/Users/kevinmaikjablonka/miniconda3/envs/gpt3/lib/pyth

In [15]:
from gpt3forchem.output import test_inverse_bandgap

/Users/kevinmaikjablonka/miniconda3/envs/gpt3/lib/python3.9/site-packages/gpflow/experimental/utils.py:42: UserWarning: You're calling gpflow.experimental.check_shapes.decorator.check_shapes which is considered *experimental*. Expect: breaking changes, poor documentation, and bugs.
  warn(
/Users/kevinmaikjablonka/miniconda3/envs/gpt3/lib/python3.9/site-packages/gpflow/experimental/utils.py:42: UserWarning: You're calling gpflow.experimental.check_shapes.inheritance.inherit_check_shapes which is considered *experimental*. Expect: breaking changes, poor documentation, and bugs.
  warn(


In [16]:
def test_inverse_model(
    modelname,
    test_prompts,
    df_train,
    max_tokens: int = 250,
    temperatures=None,
    representation="SMILES",
):
    temperatures = temperatures or [0, 0.25, 0.5, 0.75, 1.0, 1.25, 1.5]
    train_smiles = df_train["SMILES"].to_list()
    results = []
    for temperature in temperatures:
        try:
            print(f"Testing temperature {temperature} for {representation}")
            result = test_inverse_bandgap(
                test_prompts,
                modelname,
                train_smiles=train_smiles,
                temperature=temperature,
                max_tokens=max_tokens,
                representation=representation,
            )

            results.append(result)
        except Exception as e:
            print(e)
            pass

    return results

In [17]:
res_500_smiles = test_inverse_model(
    modelname_small_smiles,
    test_prompts_small_smiles,
    train_prompts_small_smiles,
    representation="SMILES",
)

Testing temperature 0 for SMILES


2022-11-27 22:15:59.803 | DEBUG    | gpt3forchem.output:test_inverse_bandgap:928 - Got predictions, example: [H]c1c([H])c([H])c(C([H])([H])N([H])C(=O)c2c([H])c([H])c(C(=O)N([H])c3c([H])c([H])c([H])c([H])c3[H])c([H])c2[H])c([H])c1[H]
2022-11-27 22:15:59.804 | DEBUG    | gpt3forchem.output:test_inverse_bandgap:939 - Loaded predictions. Example: [H]c1c([H])c([H])c(C([H])([H])N([H])C(=O)c2c([H])c([H])c(C(=O)N([H])c3c([H])c([H])c([H])c([H])c3[H])c([H])c2[H])c([H])c1[H]
2022-11-27 22:15:59.826 | DEBUG    | gpt3forchem.output:test_inverse_bandgap:950 - Calculating Frechet ChemNet distance for 100 samples
2022-11-27 22:15:59.879 | INFO     | gpt3forchem.output:_load_chemnet:217 - Saved ChemNet model to '/var/folders/m9/_txh68y946s4pxy1x2wnd3lh0000gn/T/ChemNet_v0.13_pretrained.h5'
2022-11-27 22:16:00.773331: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-11-27 22:16:02.143 | DEBUG    | gpt3forchem.output:test_inverse_bandgap:959 - Computed frec

Testing temperature 0.25 for SMILES


2022-11-27 22:16:37.239 | DEBUG    | gpt3forchem.output:test_inverse_bandgap:928 - Got predictions, example: [H]c1c([H])c(C([H])([H])N([H])C(=O)c2c([H])c([H])c(C(=O)N([H])c3c([H])c([H])c([H])c([H])c3[H])c([H])c2[H])c([H])c([H])c1Cl
2022-11-27 22:16:37.243 | DEBUG    | gpt3forchem.output:test_inverse_bandgap:939 - Loaded predictions. Example: [H]c1c([H])c(C([H])([H])N([H])C(=O)c2c([H])c([H])c(C(=O)N([H])c3c([H])c([H])c([H])c([H])c3[H])c([H])c2[H])c([H])c([H])c1Cl
2022-11-27 22:16:37.281 | DEBUG    | gpt3forchem.output:test_inverse_bandgap:950 - Calculating Frechet ChemNet distance for 76 samples
2022-11-27 22:16:37.340 | INFO     | gpt3forchem.output:_load_chemnet:217 - Saved ChemNet model to '/var/folders/m9/_txh68y946s4pxy1x2wnd3lh0000gn/T/ChemNet_v0.13_pretrained.h5'
2022-11-27 22:16:39.339 | DEBUG    | gpt3forchem.output:test_inverse_bandgap:959 - Computed frechet score: (57.71813835865383, 9.69764319338719e-06)
2022-11-27 22:16:39.790 | DEBUG    | gpt3forchem.output:test_inverse_ba

Testing temperature 0.5 for SMILES


...

2022-11-27 22:18:29.214 | DEBUG    | gpt3forchem.output:test_inverse_bandgap:928 - Got predictions, example: [H]Oc1c([H])c(C([H])([H])[H])c([H])c(N([H])C(=O)c2c([H])c([H])c(C(F)(F)F)c([H])c2C([H])([H])[H])c1[H]
2022-11-27 22:18:29.215 | DEBUG    | gpt3forchem.output:test_inverse_bandgap:939 - Loaded predictions. Example: [H]Oc1c([H])c(C([H])([H])[H])c([H])c(N([H])C(=O)c2c([H])c([H])c(C(F)(F)F)c([H])c2C([H])([H])[H])c1[H]
2022-11-27 22:18:29.257 | DEBUG    | gpt3forchem.output:test_inverse_bandgap:950 - Calculating Frechet ChemNet distance for 18 samples
2022-11-27 22:18:29.285 | INFO     | gpt3forchem.output:_load_chemnet:217 - Saved ChemNet model to '/var/folders/m9/_txh68y946s4pxy1x2wnd3lh0000gn/T/ChemNet_v0.13_pretrained.h5'
2022-11-27 22:18:31.028 | DEBUG    | gpt3forchem.output:test_inverse_bandgap:959 - Computed frechet score: (50.68632613408735, 3.957688877697366e-05)
2022-11-27 22:18:31.168 | DEBUG    | gpt3forchem.output:test_inverse_bandgap:970 - Computed KL div score: 0.5249

Testing temperature 1.25 for SMILES


2022-11-27 22:19:06.559 | DEBUG    | gpt3forchem.output:test_inverse_bandgap:928 - Got predictions, example: [H]C=C#Cc1c([H])c([H])c(C([H])([H])n2c(C([H])([H])S(=O)(=O)c3c([H])cards(C($Figators)F)c(Cl)c3[H])c([H])c(OC([H])([H])[H])c2[H])c1C#N[H]
2022-11-27 22:19:06.561 | DEBUG    | gpt3forchem.output:test_inverse_bandgap:939 - Loaded predictions. Example: [H]C=C#Cc1c([H])c([H])c(C([H])([H])n2c(C([H])([H])S(=O)(=O)c3c([H])cards(C($Figators)F)c(Cl)c3[H])c([H])c(OC([H])([H])[H])c2[H])c1C#N[H]
2022-11-27 22:19:06.609 | DEBUG    | gpt3forchem.output:test_inverse_bandgap:950 - Calculating Frechet ChemNet distance for 9 samples
2022-11-27 22:19:06.651 | INFO     | gpt3forchem.output:_load_chemnet:217 - Saved ChemNet model to '/var/folders/m9/_txh68y946s4pxy1x2wnd3lh0000gn/T/ChemNet_v0.13_pretrained.h5'
2022-11-27 22:19:08.219 | DEBUG    | gpt3forchem.output:test_inverse_bandgap:959 - Computed frechet score: (64.34800037696363, 2.5751569470566044e-06)
2022-11-27 22:19:08.307 | DEBUG    | gpt3f

Testing temperature 1.5 for SMILES


2022-11-27 22:19:44.011 | DEBUG    | gpt3forchem.output:test_inverse_bandgap:928 - Got predictions, example: [hvin]),n(-nc(-c2c([H])n(-Ct1c([H])c([H])c(-c4c([H])c([H])ncmedium5n(N5C([H])([H])C([H])([H])C5([H])C([H])([H])C([H])([H])C6
2022-11-27 22:19:44.012 | DEBUG    | gpt3forchem.output:test_inverse_bandgap:939 - Loaded predictions. Example: [hvin]),n(-nc(-c2c([H])n(-Ct1c([H])c([H])c(-c4c([H])c([H])ncmedium5n(N5C([H])([H])C([H])([H])C5([H])C([H])([H])C([H])([H])C6
2022-11-27 22:19:44.054 | DEBUG    | gpt3forchem.output:test_inverse_bandgap:950 - Calculating Frechet ChemNet distance for 3 samples
2022-11-27 22:19:44.082 | INFO     | gpt3forchem.output:_load_chemnet:217 - Saved ChemNet model to '/var/folders/m9/_txh68y946s4pxy1x2wnd3lh0000gn/T/ChemNet_v0.13_pretrained.h5'
2022-11-27 22:19:45.991 | DEBUG    | gpt3forchem.output:test_inverse_bandgap:959 - Computed frechet score: (90.77580457366079, 1.304108713424753e-08)
2022-11-27 22:19:46.026 | DEBUG    | gpt3forchem.output:test_invers

In [19]:
timestamp = time.strftime('%Y-%m-%d-%H-%M-%S', time.localtime())

In [155]:
save_pickle(f"run_files/{timestamp}-res_500_smiles.pkl", res_500_smiles)

In [30]:
smiles_500_sets = []

for res in res_500_smiles:
    result = {}
    result["temperature"] = res["meta"]["temperature"]
    result["smiles"] = set(
        res["predictions"][res["valid_smiles"]]
    )
    result['original_prediction_indices'] = [i for i, x in enumerate(res["predictions"]) if x in result['smiles']]
    result['expected'] = [res['expectations'][i] for i in result['original_prediction_indices']]
    smiles_500_sets.append(result)

for res in smiles_500_sets:
    temp = res["temperature"]
    smiles_set = res["smiles"]
    with open(f"for_xtb_opt/{timestamp}500_smiles_{temp}.txt", "w") as f:
        for i, smiles in enumerate(smiles_set):
            if i != len(smiles_set) - 1:
                f.write(smiles + "\n")
            else:
                f.write(smiles)

For the ground truth, we need to swtich to running on the server


In [23]:
from natsort import natsorted

In [119]:
def get_xtb_results(filename, results):
    outputs = []
    for res in results:
        temp = res["temperature"]
        with open(f'for_xtb_opt/{filename}_{temp}_gaps.txt', 'r') as f:
            gaps_ = f.readlines()
            if not gaps_: print(f"No results found for {temp}"); continue
            gaps = []
            for _gap in gaps_:
                name, gap = _gap.split()
                gaps.append((int(name.split("/")[-1]), float(gap) if gap !='None' else np.nan))

        predicted, true = [], []

        for i, g in gaps:
            predicted.append(g)
            true.append(res["expected"][i])

        true, predicted = np.array(true), np.array(predicted)

        not_nan_mask = ~np.isnan(predicted)
        outputs.append(
            {
                'predicted': predicted,
                'true': true,
                'temperature': temp,
                 'r2_score': r2_score(true[not_nan_mask], predicted[not_nan_mask]),
                'mae': mean_absolute_error(true[not_nan_mask], predicted[not_nan_mask]),
            }
        )
    return outputs

In [156]:
smiles_small_xtb_results = get_xtb_results('2022-11-27-22-29-43500_smiles', smiles_500_sets)
save_pickle('run_files/2022-11-27-22-29-43500_smiles_xtb_results.pkl', smiles_small_xtb_results)

#### Now use SELFIES


In [109]:
train_prompts_selfies, test_prompts_selfies = generate_inverse_photoswitch_prompts(
    train_data_small, "selfies"
), generate_inverse_photoswitch_prompts(test_data_small, "selfies")

In [112]:
modelname_500_selfies, train_filename_500_selfies = train_inverse_model(
    train_prompts_selfies, "selfies"
)

Traceback (most recent call last):
  File "/Users/kevinmaikjablonka/miniconda3/envs/gpt3/bin/openai", line 8, in <module>
    sys.exit(main())
  File "/Users/kevinmaikjablonka/miniconda3/envs/gpt3/lib/python3.9/site-packages/openai/_openai_scripts.py", line 63, in main
    args.func(args)
  File "/Users/kevinmaikjablonka/miniconda3/envs/gpt3/lib/python3.9/site-packages/openai/cli.py", line 545, in sync
    resp = openai.wandb_logger.WandbLogger.sync(
  File "/Users/kevinmaikjablonka/miniconda3/envs/gpt3/lib/python3.9/site-packages/openai/wandb_logger.py", line 74, in sync
    fine_tune_logged = [
  File "/Users/kevinmaikjablonka/miniconda3/envs/gpt3/lib/python3.9/site-packages/openai/wandb_logger.py", line 75, in <listcomp>
    cls._log_fine_tune(
  File "/Users/kevinmaikjablonka/miniconda3/envs/gpt3/lib/python3.9/site-packages/openai/wandb_logger.py", line 125, in _log_fine_tune
    wandb_run = cls._get_wandb_run(run_path)
  File "/Users/kevinmaikjablonka/miniconda3/envs/gpt3/lib/pyth

In [137]:
res_500_seflies = test_inverse_model(
    modelname_500_selfies,
    test_prompts_selfies,
    train_prompts_selfies,
    representation="selfies",
    max_tokens=1600,
    temperatures=[0]
)

Testing temperature 0 for selfies


2022-11-28 09:42:25.264 | DEBUG    | gpt3forchem.output:test_inverse_bandgap:928 - Got predictions, example: [H][C][=C][Branch1][C][H][C][Branch1][C][H][=C][Branch2][Branch2][=Branch2][C][Branch1][C][H][Branch1][C][H][N][Branch1][C][H][C][=Branch1][C][=O][C][Branch1][C][H][Branch1][C][H][C][Branch1][C][H][Branch1][C][H][C][Branch1][C][H][Branch1][C][H][C][Branch1][C][H][Branch1][C][H][C][Branch1][C][H][Branch1][C][H][C][Branch1][C][H][Branch1][C][H][C][Branch1][C][H][Branch1][C][H][C][Branch1][C][H][Branch1][C][H][C][Branch1][C][H][Branch1][C][H][C][Branch1][C][H][Branch1][C][H][C][Branch1][C][H][Branch1][C][H][C][Branch1][C][H][Branch1][C][H][C][Branch1][C][H][Branch1][C][H][C][Branch1][C][H][Branch1][C][H][C][Branch1][C][H][Branch1][C][H][C][Branch1][C][H][Branch1][C][H][C][Branch1][C][H][Branch1][C][H][C][Branch1][C][H][Branch1][C][H][C][Branch1][C][H][Branch1][C][H][C][Branch1][C][H][Branch1][C][H][C][Branch1][C][H][Branch1][C][H][C][Branch1][C][H][Branch1][C][H][C][Branch1][C][H][

In [122]:
save_pickle(f"run_files/{timestamp}-res_500_selfies.pkl", res_500_seflies)

In [134]:
res_500_seflies[0]['predictions']

array(['[H][C][=C][Branch1][C][H][C][Branch1][C][H][=C][Branch2][Branch2][=Branch2][C][Branch1][C][H][Branch1][C][H][N][Branch1][C][H][C][=Branch1][C][=O][C][Branch1][C][H][Branch1][C][H][C][Branch1][C][H][Branch1][C][H][C][Branch1][C][H][Branch1][C][H][C][Branch1][C][H][Branch1][C][H][C][Branch1][C][H][Branch1][C][H][C][Branch1][C][H][Branch1][C][H][C][Branch1][C][H][Branch1][C][H][C][Branch1][C][H][Branch1][C][H][C][Branch1][C][H][Branch1][C][H][C][Branch1][',
       '[H][C][=C][Branch1][C][H][C][Branch1][C][H][=C][Branch2][Branch2][=Branch2][C][Branch1][C][H][Branch1][C][H][N][Branch1][C][H][C][=Branch1][C][=O][C][Branch1][C][H][Branch1][C][H][C][Branch1][C][H][Branch1][C][H][C][Branch1][C][H][Branch1][C][H][C][Branch1][C][H][Branch1][C][H][C][Branch1][C][H][Branch1][C][H][C][Branch1][C][H][Branch1][C][H][C][Branch1][C][H][Branch1][C][H][C][Branch1][C][H][Branch1][C][H][C][Branch1][C][H][Branch1][C][H][C][Branch1][',
       '[H][C][=C][Branch1][C][H][C][Branch1][C][H][=C][Branch2][B

In [124]:
import selfies as sf

In [132]:
selfies_500_sets = []

for res in res_500_seflies:
    result = {}
    result["temperature"] = res["meta"]["temperature"]
    print(res["predictions"][res["valid_smiles"]])
    result["selfies"] = set(
        res["predictions"][res["valid_smiles"]]
    )
    print(result["selfies"])
    result['smiles'] = [sf.decoder(x) for x in result['selfies']]
    result['original_prediction_indices'] = [i for i, x in enumerate(res["predictions"]) if x in result['selfies']]
    result['expected'] = [res['expectations'][i] for i in result['original_prediction_indices']]
    selfies_500_sets.append(result)

for res in selfies_500_sets:
    temp = res["temperature"]
    smiles_set = res["smiles"]
    with open(f"for_xtb_opt/{timestamp}500_selfies_{temp}.txt", "w") as f:
        for i, smiles in enumerate(smiles_set):
            if i != len(smiles_set) - 1:
                f.write(smiles + "\n")
            else:
                f.write(smiles)

[]
set()
['[H]C=NC(N([H])C(=O)N([H])C1=C([H])C([H])=C([H])C([H])=C1C([H])[H])[H]'
 '[H]C=NC(C1=C([H])C([H])=C([H])C([H])=C1[H])[H]'
 '[H]C=C([H])CC=C([H])C([H])=C(C1=C([H])C([H])=C([H])C([H])=C1[H])C[H]'
 '[H]OC(=O)C=C([H])C([H])=CC1=C([H])C([H])=C([H])C([H])=C1[H]']
{'[H]C=NC(N([H])C(=O)N([H])C1=C([H])C([H])=C([H])C([H])=C1C([H])[H])[H]', '[H]C=NC(C1=C([H])C([H])=C([H])C([H])=C1[H])[H]', '[H]C=C([H])CC=C([H])C([H])=C(C1=C([H])C([H])=C([H])C([H])=C1[H])C[H]', '[H]OC(=O)C=C([H])C([H])=CC1=C([H])C([H])=C([H])C([H])=C1[H]'}


DecoderError: invalid symbol 'C=NC(N([H]'
	SELFIES: [H]C=NC(N([H])C(=O)N([H])C1=C([H])C([H])=C([H])C([H])=C1C([H])[H])[H]

## Larger training set


In [138]:
train_large, test_large = train_test_split(
    data, train_size=1000, test_size=100, random_state=42
)

In [139]:
(
    train_prompts_large_smiles,
    test_prompts_large_smiles,
) = generate_inverse_photoswitch_prompts(
    train_large, "smiles"
), generate_inverse_photoswitch_prompts(
    test_large, "smiles"
)

(
    train_prompts_large_selfies,
    test_prompts_large_selfies,
) = generate_inverse_photoswitch_prompts(
    train_large, "selfies"
), generate_inverse_photoswitch_prompts(
    test_large, "selfies"
)

In [140]:
modelname_1000_smiles, train_filename_1000_smiles = train_inverse_model(
    train_prompts_large_smiles, "smiles"
)

Traceback (most recent call last):
  File "/Users/kevinmaikjablonka/miniconda3/envs/gpt3/bin/openai", line 8, in <module>
    sys.exit(main())
  File "/Users/kevinmaikjablonka/miniconda3/envs/gpt3/lib/python3.9/site-packages/openai/_openai_scripts.py", line 63, in main
    args.func(args)
  File "/Users/kevinmaikjablonka/miniconda3/envs/gpt3/lib/python3.9/site-packages/openai/cli.py", line 545, in sync
    resp = openai.wandb_logger.WandbLogger.sync(
  File "/Users/kevinmaikjablonka/miniconda3/envs/gpt3/lib/python3.9/site-packages/openai/wandb_logger.py", line 74, in sync
    fine_tune_logged = [
  File "/Users/kevinmaikjablonka/miniconda3/envs/gpt3/lib/python3.9/site-packages/openai/wandb_logger.py", line 75, in <listcomp>
    cls._log_fine_tune(
  File "/Users/kevinmaikjablonka/miniconda3/envs/gpt3/lib/python3.9/site-packages/openai/wandb_logger.py", line 125, in _log_fine_tune
    wandb_run = cls._get_wandb_run(run_path)
  File "/Users/kevinmaikjablonka/miniconda3/envs/gpt3/lib/pyth

In [141]:
res_1000_smiles = test_inverse_model(
    modelname_1000_smiles,
    test_prompts_large_smiles,
    train_prompts_large_smiles,
    representation="SMILES",
)

Testing temperature 0 for SMILES


2022-11-28 10:15:23.805 | DEBUG    | gpt3forchem.output:test_inverse_bandgap:928 - Got predictions, example: [H]c1c([H])c([H])c(C([H])([H])N2C([H])([H])C([H])([H])N(C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H
2022-11-28 10:15:23.806 | DEBUG    | gpt3forchem.output:test_inverse_bandgap:939 - Loaded predictions. Example: [H]c1c([H])c([H])c(C([H])([H])N2C([H])([H])C([H])([H])N(C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H
2022-11-28 10:15:23.831 | DEBUG    | gpt3forchem.output:test_inverse_bandgap:950 - Calculating Fr

Testing temperature 0.25 for SMILES


2022-11-28 10:15:58.877 | DEBUG    | gpt3forchem.output:test_inverse_bandgap:928 - Got predictions, example: [H]c1c([H])c([H])c(C([H])([H])[C@@]([H])(C(=O)N([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([
2022-11-28 10:15:58.879 | DEBUG    | gpt3forchem.output:test_inverse_bandgap:939 - Loaded predictions. Example: [H]c1c([H])c([H])c(C([H])([H])[C@@]([H])(C(=O)N([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([
2022-11-28 10:15:58.913 | DEBUG    | gpt3forchem.output:test_inverse_bandgap:950 - Calculating 

Testing temperature 0.5 for SMILES


...

2022-11-28 10:17:49.717 | DEBUG    | gpt3forchem.output:test_inverse_bandgap:928 - Got predictions, example: [H]c1nc([C@]2([H])C([H])([H])C([H])([H])N(C([H])([H])c3nc([H])c([N+](=O)[O-])c([H])c3[H])C([H])([H])C([H])([H])S2([H])[H])nc([H])c1OC([H])([H])C([H])([H])C([H])([H])C2([H])[H]
2022-11-28 10:17:49.718 | DEBUG    | gpt3forchem.output:test_inverse_bandgap:939 - Loaded predictions. Example: [H]c1nc([C@]2([H])C([H])([H])C([H])([H])N(C([H])([H])c3nc([H])c([N+](=O)[O-])c([H])c3[H])C([H])([H])C([H])([H])S2([H])[H])nc([H])c1OC([H])([H])C([H])([H])C([H])([H])C2([H])[H]
2022-11-28 10:17:49.768 | DEBUG    | gpt3forchem.output:test_inverse_bandgap:950 - Calculating Frechet ChemNet distance for 28 samples
2022-11-28 10:17:49.796 | INFO     | gpt3forchem.output:_load_chemnet:217 - Saved ChemNet model to '/var/folders/m9/_txh68y946s4pxy1x2wnd3lh0000gn/T/ChemNet_v0.13_pretrained.h5'
2022-11-28 10:17:51.123 | DEBUG    | gpt3forchem.output:test_inverse_bandgap:959 - Computed frechet score: (39.452

Testing temperature 1.25 for SMILES


2022-11-28 10:18:27.444 | DEBUG    | gpt3forchem.output:test_inverse_bandgap:928 - Got predictions, example: [H]c1sc(-c2oc3c([H])c([H])c([H])c([H])c3c2n2C([H])([H])C([H])([H])O3)c([H])c(C(=O)OC([H])([H])c2c([H])c([H])c([H])c([H])c2[H])c1[H]
2022-11-28 10:18:27.445 | DEBUG    | gpt3forchem.output:test_inverse_bandgap:939 - Loaded predictions. Example: [H]c1sc(-c2oc3c([H])c([H])c([H])c([H])c3c2n2C([H])([H])C([H])([H])O3)c([H])c(C(=O)OC([H])([H])c2c([H])c([H])c([H])c([H])c2[H])c1[H]
2022-11-28 10:18:27.472 | DEBUG    | gpt3forchem.output:test_inverse_bandgap:950 - Calculating Frechet ChemNet distance for 7 samples
2022-11-28 10:18:27.515 | INFO     | gpt3forchem.output:_load_chemnet:217 - Saved ChemNet model to '/var/folders/m9/_txh68y946s4pxy1x2wnd3lh0000gn/T/ChemNet_v0.13_pretrained.h5'
2022-11-28 10:18:33.032 | DEBUG    | gpt3forchem.output:test_inverse_bandgap:959 - Computed frechet score: (65.18249915247344, 2.1793152620181415e-06)
2022-11-28 10:18:33.098 | DEBUG    | gpt3forchem.out

Testing temperature 1.5 for SMILES


2022-11-28 10:19:11.079 | DEBUG    | gpt3forchem.output:test_inverse_bandgap:928 - Got predictions, example: [H]ON1C(=O)c2sc(=o)c([H])c([H])c2C1=O
2022-11-28 10:19:11.079 | DEBUG    | gpt3forchem.output:test_inverse_bandgap:939 - Loaded predictions. Example: [H]ON1C(=O)c2sc(=o)c([H])c([H])c2C1=O
2022-11-28 10:19:11.103 | DEBUG    | gpt3forchem.output:test_inverse_bandgap:950 - Calculating Frechet ChemNet distance for 3 samples
2022-11-28 10:19:11.176 | INFO     | gpt3forchem.output:_load_chemnet:217 - Saved ChemNet model to '/var/folders/m9/_txh68y946s4pxy1x2wnd3lh0000gn/T/ChemNet_v0.13_pretrained.h5'
2022-11-28 10:19:17.189 | DEBUG    | gpt3forchem.output:test_inverse_bandgap:959 - Computed frechet score: (100.93881271193972, 1.708307807952628e-09)
2022-11-28 10:19:17.216 | DEBUG    | gpt3forchem.output:test_inverse_bandgap:970 - Computed KL div score: 0.5275411763797172


In [143]:
save_pickle(f"run_files/{timestamp}-res_1000_smiles.pkl", res_1000_smiles)

In [144]:
smiles_1000_sets = []

for res in res_1000_smiles:
    result = {}
    result["temperature"] = res["meta"]["temperature"]
    result["smiles"] = set(
        res["predictions"][res["valid_smiles"]]
    )
    result['original_prediction_indices'] = [i for i, x in enumerate(res["predictions"]) if x in result['smiles']]
    result['expected'] = [res['expectations'][i] for i in result['original_prediction_indices']]
    smiles_1000_sets.append(result)

for res in smiles_1000_sets:
    temp = res["temperature"]
    smiles_set = res["smiles"]
    with open(f"for_xtb_opt/{timestamp}1000_smiles_{temp}.txt", "w") as f:
        for i, smiles in enumerate(smiles_set):
            if i != len(smiles_set) - 1:
                f.write(smiles + "\n")
            else:
                f.write(smiles)

In [157]:
smiles_large_xtb_results = get_xtb_results('2022-11-27-22-29-431000_smiles', smiles_1000_sets)
save_pickle('run_files/2022-11-27-22-29-431000_smiles_xtb_results.pkl', smiles_large_xtb_results)

No results found for 0


## Very large

In [149]:
train_very_large, test_very_large = train_test_split(
    data, train_size=5000, test_size=100, random_state=42
)

In [150]:
(
    train_prompts_very_large_smiles,
    test_prompts_very_large_smiles,
) = generate_inverse_photoswitch_prompts(
    train_very_large, "smiles"
), generate_inverse_photoswitch_prompts(
    test_very_large, "smiles"
)

(
    train_prompts_very_large_selfies,
    test_prompts_very_large_selfies,
) = generate_inverse_photoswitch_prompts(
    train_very_large, "selfies"
), generate_inverse_photoswitch_prompts(
    test_very_large, "selfies"
)

In [151]:
modelname_5000_smiles, train_filename_5000_smiles = train_inverse_model(
    train_prompts_very_large_smiles, "smiles"
)

Traceback (most recent call last):
  File "/Users/kevinmaikjablonka/miniconda3/envs/gpt3/bin/openai", line 8, in <module>
    sys.exit(main())
  File "/Users/kevinmaikjablonka/miniconda3/envs/gpt3/lib/python3.9/site-packages/openai/_openai_scripts.py", line 63, in main
    args.func(args)
  File "/Users/kevinmaikjablonka/miniconda3/envs/gpt3/lib/python3.9/site-packages/openai/cli.py", line 545, in sync
    resp = openai.wandb_logger.WandbLogger.sync(
  File "/Users/kevinmaikjablonka/miniconda3/envs/gpt3/lib/python3.9/site-packages/openai/wandb_logger.py", line 74, in sync
    fine_tune_logged = [
  File "/Users/kevinmaikjablonka/miniconda3/envs/gpt3/lib/python3.9/site-packages/openai/wandb_logger.py", line 75, in <listcomp>
    cls._log_fine_tune(
  File "/Users/kevinmaikjablonka/miniconda3/envs/gpt3/lib/python3.9/site-packages/openai/wandb_logger.py", line 125, in _log_fine_tune
    wandb_run = cls._get_wandb_run(run_path)
  File "/Users/kevinmaikjablonka/miniconda3/envs/gpt3/lib/pyth

In [152]:
res_5000_smiles = test_inverse_model(
    modelname_5000_smiles,
    test_prompts_very_large_smiles,
    train_prompts_very_large_smiles,
    representation="SMILES",
)

Testing temperature 0 for SMILES


2022-11-28 12:55:08.387 | DEBUG    | gpt3forchem.output:test_inverse_bandgap:928 - Got predictions, example: [H]c1c([H])c([H])c(C([H])([H])N([H])C(=O)N([H])c2c([H])c([H])c(C([H])([H])[H])c([H])c2[H])c([H])c1[H]
2022-11-28 12:55:08.387 | DEBUG    | gpt3forchem.output:test_inverse_bandgap:939 - Loaded predictions. Example: [H]c1c([H])c([H])c(C([H])([H])N([H])C(=O)N([H])c2c([H])c([H])c(C([H])([H])[H])c([H])c2[H])c([H])c1[H]
2022-11-28 12:55:08.441 | DEBUG    | gpt3forchem.output:test_inverse_bandgap:950 - Calculating Frechet ChemNet distance for 49 samples
2022-11-28 12:55:08.530 | INFO     | gpt3forchem.output:_load_chemnet:217 - Saved ChemNet model to '/var/folders/m9/_txh68y946s4pxy1x2wnd3lh0000gn/T/ChemNet_v0.13_pretrained.h5'
2022-11-28 12:55:10.229 | DEBUG    | gpt3forchem.output:test_inverse_bandgap:959 - Computed frechet score: (68.56659011537226, 1.1075967022091465e-06)
2022-11-28 12:55:10.473 | DEBUG    | gpt3forchem.output:test_inverse_bandgap:970 - Computed KL div score: 0.070

Testing temperature 0.25 for SMILES


2022-11-28 12:55:45.286 | DEBUG    | gpt3forchem.output:test_inverse_bandgap:928 - Got predictions, example: [H]c1c([H])c([H])c(C([H])([H])N2C([H])([H])C([H])([H])C([H])(C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H]
2022-11-28 12:55:45.287 | DEBUG    | gpt3forchem.output:test_inverse_bandgap:939 - Loaded predictions. Example: [H]c1c([H])c([H])c(C([H])([H])N2C([H])([H])C([H])([H])C([H])(C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H]
2022-11-28 12:55:45.328 | DEBUG    | gpt3forchem.output:test_inverse_bandgap:950 - Calculating 

Testing temperature 0.5 for SMILES


...

2022-11-28 12:58:15.636 | DEBUG    | gpt3forchem.output:test_inverse_bandgap:928 - Got predictions, example: [H]OC(=O)[C@@]([H])(N([H])S(=O)(=O)c1c([H])c([H])c([H])c(C([H])([H])N2c3c4c([H])c([H])c([H])c([H])c4n(C([H])(C([H])([H])[H])C([H])([H])[H])c3c(=O)c2[H])c1[H])C([H])([H])c1c([H])c([H])c(OC([H])([H])[H])c(C([H])([H])[H])c1[H]
2022-11-28 12:58:15.637 | DEBUG    | gpt3forchem.output:test_inverse_bandgap:939 - Loaded predictions. Example: [H]OC(=O)[C@@]([H])(N([H])S(=O)(=O)c1c([H])c([H])c([H])c(C([H])([H])N2c3c4c([H])c([H])c([H])c([H])c4n(C([H])(C([H])([H])[H])C([H])([H])[H])c3c(=O)c2[H])c1[H])C([H])([H])c1c([H])c([H])c(OC([H])([H])[H])c(C([H])([H])[H])c1[H]
2022-11-28 12:58:15.691 | DEBUG    | gpt3forchem.output:test_inverse_bandgap:950 - Calculating Frechet ChemNet distance for 64 samples
2022-11-28 12:58:15.748 | INFO     | gpt3forchem.output:_load_chemnet:217 - Saved ChemNet model to '/var/folders/m9/_txh68y946s4pxy1x2wnd3lh0000gn/T/ChemNet_v0.13_pretrained.h5'
2022-11-28 12:58:1

Testing temperature 1.25 for SMILES


2022-11-28 12:58:53.863 | DEBUG    | gpt3forchem.output:test_inverse_bandgap:928 - Got predictions, example: [H]c1c([H])c([H])c2nsc(N([H])C([H])([H])C#N)c2c1[H]
2022-11-28 12:58:53.864 | DEBUG    | gpt3forchem.output:test_inverse_bandgap:939 - Loaded predictions. Example: [H]c1c([H])c([H])c2nsc(N([H])C([H])([H])C#N)c2c1[H]
2022-11-28 12:58:53.911 | DEBUG    | gpt3forchem.output:test_inverse_bandgap:950 - Calculating Frechet ChemNet distance for 31 samples
2022-11-28 12:58:53.987 | INFO     | gpt3forchem.output:_load_chemnet:217 - Saved ChemNet model to '/var/folders/m9/_txh68y946s4pxy1x2wnd3lh0000gn/T/ChemNet_v0.13_pretrained.h5'
2022-11-28 12:58:55.463 | DEBUG    | gpt3forchem.output:test_inverse_bandgap:959 - Computed frechet score: (41.40899478572808, 0.0002530815074856264)
2022-11-28 12:58:55.649 | DEBUG    | gpt3forchem.output:test_inverse_bandgap:970 - Computed KL div score: 0.6509537496104156


Testing temperature 1.5 for SMILES


2022-11-28 12:59:31.561 | DEBUG    | gpt3forchem.output:test_inverse_bandgap:928 - Got predictions, example: [H]OP(C(=O)N1c2nn(-c3c([H])c([H])c([H])c(OC([H])([H])[H])c3C#N)[c2N(C([H])([H])c2nc3c([H])c([H])c(#N)c([H])c3u2)SC([H])([H])[C@@]1(C(=O)N([H])[H])C([H])([H])C([H])(C([H])([H])[H])C([H])([H])[H]
2022-11-28 12:59:31.561 | DEBUG    | gpt3forchem.output:test_inverse_bandgap:939 - Loaded predictions. Example: [H]OP(C(=O)N1c2nn(-c3c([H])c([H])c([H])c(OC([H])([H])[H])c3C#N)[c2N(C([H])([H])c2nc3c([H])c([H])c(#N)c([H])c3u2)SC([H])([H])[C@@]1(C(=O)N([H])[H])C([H])([H])C([H])(C([H])([H])[H])C([H])([H])[H]
2022-11-28 12:59:31.587 | DEBUG    | gpt3forchem.output:test_inverse_bandgap:950 - Calculating Frechet ChemNet distance for 2 samples
2022-11-28 12:59:31.659 | INFO     | gpt3forchem.output:_load_chemnet:217 - Saved ChemNet model to '/var/folders/m9/_txh68y946s4pxy1x2wnd3lh0000gn/T/ChemNet_v0.13_pretrained.h5'
2022-11-28 12:59:34.037 | DEBUG    | gpt3forchem.output:test_inverse_bandgap:95

In [153]:
save_pickle(f"run_files/{timestamp}-res_5000_smiles.pkl", res_5000_smiles)

In [154]:
smiles_5000_sets = []

for res in res_5000_smiles:
    result = {}
    result["temperature"] = res["meta"]["temperature"]
    result["smiles"] = set(
        res["predictions"][res["valid_smiles"]]
    )
    result['original_prediction_indices'] = [i for i, x in enumerate(res["predictions"]) if x in result['smiles']]
    result['expected'] = [res['expectations'][i] for i in result['original_prediction_indices']]
    smiles_5000_sets.append(result)

for res in smiles_5000_sets:
    temp = res["temperature"]
    smiles_set = res["smiles"]
    with open(f"for_xtb_opt/{timestamp}5000_smiles_{temp}.txt", "w") as f:
        for i, smiles in enumerate(smiles_set):
            if i != len(smiles_set) - 1:
                f.write(smiles + "\n")
            else:
                f.write(smiles)

In [159]:
smiles_very_large_xtb_results = get_xtb_results('2022-11-27-22-29-435000_smiles', smiles_5000_sets)
save_pickle('run_files/2022-11-27-22-29-435000_smiles_xtb_results.pkl', smiles_very_large_xtb_results)

FileNotFoundError: [Errno 2] No such file or directory: 'for_xtb_opt/2022-11-27-22-29-435000_smiles_0_gaps.txt'

## Very very large dataset

In [161]:
train_very_very_large, test_very_very_large = train_test_split(
    data, train_size=50_000, test_size=100, random_state=42
)

In [162]:
(
    train_prompts_very_very_large_smiles,
    test_prompts_very_very_large_smiles,
) = generate_inverse_photoswitch_prompts(
    train_very_very_large, "smiles"
), generate_inverse_photoswitch_prompts(
    test_very_very_large, "smiles"
)

(
    train_prompts_very_very_large_selfies,
    test_prompts_very_very_large_selfies,
) = generate_inverse_photoswitch_prompts(
    train_very_very_large, "selfies"
), generate_inverse_photoswitch_prompts(
    test_very_very_large, "selfies"
)

In [163]:
modelname_50000_smiles, train_filename_50000_smiles = train_inverse_model(
    train_prompts_very_very_large_smiles, "smiles"
)

Traceback (most recent call last):
  File "/Users/kevinmaikjablonka/miniconda3/envs/gpt3/bin/openai", line 8, in <module>
    sys.exit(main())
  File "/Users/kevinmaikjablonka/miniconda3/envs/gpt3/lib/python3.9/site-packages/openai/_openai_scripts.py", line 63, in main
    args.func(args)
  File "/Users/kevinmaikjablonka/miniconda3/envs/gpt3/lib/python3.9/site-packages/openai/cli.py", line 545, in sync
    resp = openai.wandb_logger.WandbLogger.sync(
  File "/Users/kevinmaikjablonka/miniconda3/envs/gpt3/lib/python3.9/site-packages/openai/wandb_logger.py", line 74, in sync
    fine_tune_logged = [
  File "/Users/kevinmaikjablonka/miniconda3/envs/gpt3/lib/python3.9/site-packages/openai/wandb_logger.py", line 75, in <listcomp>
    cls._log_fine_tune(
  File "/Users/kevinmaikjablonka/miniconda3/envs/gpt3/lib/python3.9/site-packages/openai/wandb_logger.py", line 125, in _log_fine_tune
    wandb_run = cls._get_wandb_run(run_path)
  File "/Users/kevinmaikjablonka/miniconda3/envs/gpt3/lib/pyth